In [ ]:
!conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y
!conda install '/kaggle/input/pydicom-conda-helper/efficientnet-1.0.0-py37h06a4308_0.tar.bz2' -y
!conda install '/kaggle/input/pydicom-conda-helper/keras-applications-1.0.8-py_1.tar.bz2' -c conda-forge -y

In [ ]:
import tensorflow as tf
import albumentations as albu
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
from tensorflow.keras.utils import to_categorical
import efficientnet as efn
import glob
from tqdm import tqdm
import pydicom
from pydicom import pixel_data_handlers
import numpy as np
import cv2
import pandas as pd
import math
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import efficientnet.tfkeras as efn
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
yolo_config = {
    # Basic
    'img_size': (608, 608, 3),
    'anchors': [66, 90, 70, 98, 71, 104, 74, 102, 81, 103, 85, 92, 91, 103, 94, 99, 98, 97],
    # (608, 608) with  data only for macula with 3355 images with both axis elongated
    'strides': [8, 16, 32],
    'xyscale': [1.2, 1.1, 1.05],

    # Training
    'iou_loss_thresh': 0.65,
    'batch_size': 8,
    'num_gpu': 1,

    # Inference
    'max_boxes': 100,
    'iou_threshold': 0.5,
    'score_threshold': 0.6,
}

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    """
    Data Generator for generating a batch in real-time to be fed to a model.
    Ideally, it should work both for classification networks and segmentation/localisation networks.
    """

    def __init__(self, list_ids, labels, return_labels=True, batch_size=32, dim=(256, 256, 3), n_classes=2,
                 shuffle_data=True, augment_data=False, visualise=None, network_type='classification',
                 anchors=None, max_boxes=100, train_on_cloud=False, major_revision=0,
                 minor_revision=0, model_log_path='./', augmentations=None, use_vasculature=False,
                 vasculature_directory=None, vasculature_extension='.png'):
        self.list_ids = list_ids  # same as annotations line. Basically, the list of the images that we will read.
        self.labels = labels
        self.return_labels = return_labels
        self.batch_size = batch_size
        self.dim = dim  # image size
        self.n_classes = n_classes
        self.augment_data = augment_data
        self.shuffle_data = shuffle_data
        self.visualise = visualise
        self.network_type = network_type
        self.on_epoch_end()
        self.anchors = np.array(yolo_config['anchors']).reshape((9, 2))  # YOLO anchors
        self.max_boxes = max_boxes
        self.train_on_cloud = train_on_cloud
        self.major_revision = major_revision
        self.minor_revision = minor_revision
        self.model_log_path = model_log_path
        self.valid_augmentations = augmentations
        self.use_vasculature = use_vasculature
        self.vasculature_directory = vasculature_directory
        self.vascuture_extension = vasculature_extension

    def on_epoch_end(self):
        """
        This function will shuffle the list of images on each epoch end
        :return:
        """
        self.indices = np.arange(len(self.list_ids))
        if self.shuffle_data:
            np.random.shuffle(self.indices)

    def __data_generation(self, list_ids_temp, list_labels_temp=None):
        """
        Generates data for a single batch containing batch_size samples as X : (n_samples, *dim) and y as (n_samples,
        num_classes) in case of classification network
        Generates data for a single batch containing batch_size samples as X: (n_samples, *dim) and y as (n_samples,
        max_boxes, 5) for segmentation network.
        """
        if self.network_type == 'classification':
            # Initialization
            X = np.empty((self.batch_size, self.dim[0], self.dim[1], self.dim[2]), dtype=np.uint8)
            y = np.empty(self.batch_size, dtype=np.uint8)

            # Generate data
            for i, ID in enumerate(list_ids_temp):
                # Store sample
                #X[i, ...] = cv2.resize(cv2.imread(ID, -1), self.dim[:2])
                my_image = cv2.resize(cv2.imread(ID, -1), (self.dim[0], self.dim[1]))
                my_shape = np.shape(my_image)
                if len(my_shape) == 2:
                    X[i, :,:,0] = my_image
                    X[i, :,:,1] = X[i, :,:,0]
                    X[i, :,:,2] = X[i, :,:,0]
                elif len(my_shape) == 3:
                    X[i, ...] = my_image

                # Store class
                y[i] = list_labels_temp[i]

            if self.augment_data:
                X = self.__augment_batch(X, y)

            return X, to_categorical(y, num_classes=self.n_classes)

        elif self.network_type == 'regression':
            # Initialization
            X = np.empty((self.batch_size, self.dim[0], self.dim[1], self.dim[2]), dtype=np.uint8)
            y = np.empty(self.batch_size, dtype=np.float32)

            # Generate data
            for i, ID in enumerate(list_ids_temp):
                # Store sample
                X[i, ...] = cv2.resize(cv2.imread(ID, -1), self.dim[:2])

                # Store class
                y[i] = list_labels_temp[i]

            if self.augment_data:
                X = self.__augment_batch(X, y)

            return X, y

        elif self.network_type == 'segmentation':
            X = np.empty((len(list_ids_temp), self.dim[0], self.dim[1], self.dim[2]), dtype=np.float32)
            y_bbox = np.empty((len(list_ids_temp), self.max_boxes, 5), dtype=np.float32)

            valid_sample = 0
            for i, ID in enumerate(list_ids_temp):
                valid, img_data, box_data = self.get_data(ID)
                if not valid:
                    X[valid_sample] = img_data
                    y_bbox[valid_sample] = box_data
                    valid_sample += 1

            X = X[:valid_sample]
            y_bbox = y_bbox[:valid_sample]

            if self.augment_data:
                X, y_bbox = self.__augment_batch(X, y_bbox)

            y_tensor, y_true_boxes_xywh = self.preprocess_true_boxes(y_bbox, self.dim[:2], self.anchors,
                                                                     self.n_classes)

            return X, y_tensor, y_true_boxes_xywh

        else:
            raise Exception('Neither type of network selected')

    def __len__(self):
        """Denotes the number of batches per epoch"""
        return int(np.floor(len(self.list_ids) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data"""

        # Generate indexes of the batch
        indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        list_ids_temp = [self.list_ids[k] for k in indices]

        if self.train_on_cloud:
            grdive_path = '/content/drive/My Drive/Yolo_Data/'
            list_ids_temp = [(grdive_path + i.split('/')[-1]).replace('\\', '/') for i in list_ids_temp]

        if self.network_type == 'classification':
            list_labels_temp = [self.labels[k] for k in indices]
            # Generate data
            X, y = self.__data_generation(list_ids_temp, list_labels_temp)

            if self.return_labels:
                return X, y
            else:
                return X

        elif self.network_type == 'regression':
            list_labels_temp = [self.labels[k] for k in indices]
            # Generate data
            X, y = self.__data_generation(list_ids_temp, list_labels_temp)

            if self.return_labels:
                return X, y
            else:
                return X

        elif self.network_type == 'segmentation':
            X, y_tensor, y_bbox = self.__data_generation(list_ids_temp=list_ids_temp)

            return [X, *y_tensor, y_bbox], np.zeros(len(list_ids_temp))

        else:
            raise Exception('Neither type of network selected.')

    def get_data(self, annotation_line):
        line = annotation_line.split()
        if self.train_on_cloud:
            # print(line[0] + ' ' + line[1])
            img_path = line[0] + ' ' + line[1]
        else:
            img_path = line[0]

        if self.use_vasculature:
            img = cv2.imread(img_path, -1)
            img_name = img_path.split('/')[-1]
            # print(img_name)
            img_extension = '.' + img_name.split('.')[-1]
            # print(img_extension)
            vascultaure_name = img_name.replace(img_extension, self.vascuture_extension)
            # print(vascultaure_name)
            vasculture_path = os.path.join(self.vasculature_directory, vascultaure_name)
            # print(vasculture_path)
            vasculture = cv2.imread(vasculture_path, -1)
            img[:, :, 0] = vasculture
            img = img[:, :, ::-1]
        else:
            #print(img_path)
            img = cv2.imread(img_path, -1)
            my_shape = np.shape(img)
            if len(my_shape) < 3:
                img = np.stack((img,) * 3, axis=-1)

            if img is None:
                print(img_path)
            else:
                img = img[:, :, ::-1]

        ih, iw = img.shape[:2]
        h, w, c = self.dim
        if self.train_on_cloud:
            boxes = np.array([np.array(list(map(float, box.split(',')))) for box in line[2:]],
                             dtype=np.float32)  # x1y1x2y2
        else:
            boxes = np.array([np.array(list(map(float, box.split(',')))) for box in line[1:]],
                             dtype=np.float32)  # x1y1x2y2
        scale_w, scale_h = w / iw, h / ih
        img = cv2.resize(img, (w, h))
        image_data = np.array(img) / 255.

        # correct boxes coordinates
        box_data = np.zeros((self.max_boxes, 5))
        if len(boxes) > 0:
            np.random.shuffle(boxes)
            boxes = boxes[:self.max_boxes]
            boxes[:, [0, 2]] = boxes[:, [0, 2]] * scale_w  # + dx
            boxes[:, [1, 3]] = boxes[:, [1, 3]] * scale_h  # + dy
            # boxes[:, [0, 2]] = np.clip(boxes[:, [0, 2]], 0, w)
            # boxes[:, [1, 3]] = np.clip(boxes[:, [1, 3]], 0, h)
            box_data[:len(boxes)] = boxes

        if np.isnan(image_data).any():
            file = open(os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                        'a+')
            file.write('\n')
            file.write('NAN value encountered in image data. Sample will be discarded.')

        if np.isnan(box_data).any():
            file = open(os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                        'a+')
            file.write('\n')
            file.write('NAN value encountered in annotation data. Sample will be discarded.')

        return (np.isnan(image_data).any() or np.isnan(box_data).any()), \
               image_data, box_data
        #return (np.isnan(image_data).any() or np.isnan(box_data).any()), \
        #       np.stack((image_data,)*3, axis=-1), box_data

    def __random_transform(self, img, annotation):
        """
        We generate the augmentations that can be applied to a set of images in a dataset
        The valid_augmentations list depends on the data that is being supplied.
        """
        if self.network_type == 'classification':

            composition = albu.Compose(transforms=self.valid_augmentations[:-1], p=self.valid_augmentations[-1])

            transformed_data = composition(image=img)

            if self.visualise:
                temp_image = np.float32(transformed_data['image'])
                img = np.float32(img)
                horizontally_stacked = np.hstack((img, temp_image))
                cv2.imshow('Transformed Image', np.uint8(horizontally_stacked))
                cv2.waitKey()

            return transformed_data['image']

        elif self.network_type == 'regression':
            composition = albu.Compose(transforms=self.valid_augmentations[:-1], p=self.valid_augmentations[-1])

            transformed_data = composition(image=img)

            if self.visualise:
                temp_image = np.float32(transformed_data['image'])
                img = np.float32(img)
                horizontally_stacked = np.hstack((img, temp_image))
                cv2.imshow('Transformed Image', np.uint8(horizontally_stacked))
                cv2.waitKey()

            return transformed_data['image']


        elif self.network_type == 'segmentation':

            temp, class_labels = [], []
            for i in range(np.shape(annotation)[0]):
                if sum(annotation[i, ...]) > 0:
                    temp.append(list(annotation[i, ...][0:-1]))
                    class_labels.append(self.labels[int(annotation[i, -1])])

            composition = albu.Compose(transforms=self.valid_augmentations[:-1],
                                       bbox_params=albu.BboxParams(format='pascal_voc', min_visibility=0.7,
                                                                   label_fields=['class_labels']),
                                       p=self.valid_augmentations[-1])
            #print(composition.transforms)
            transformed_data = composition(image=np.uint8(img * 255), bboxes=temp, class_labels=class_labels)

            if self.visualise:
                temp_image = np.float32(transformed_data['image'][:, :, ::-1])
                img = np.float32(img * 255)

                for i in range(len(transformed_data['class_labels'])):
                    start_point = (int(transformed_data['bboxes'][i][0]), int(transformed_data['bboxes'][i][1]))
                    end_point = (int(transformed_data['bboxes'][i][2]), int(transformed_data['bboxes'][i][3]))
                    temp_image = cv2.rectangle(temp_image, start_point, end_point, (0, 0, 255), 2)

                    start_point = (int(annotation[i][0]), int(annotation[i][1]))
                    end_point = (int(annotation[i][2]), int(annotation[i][3]))
                    img = cv2.rectangle(img, start_point, end_point, (0, 0, 255), 2)

                horizontally_stacked = np.hstack((img[:, :, ::-1], temp_image))
                cv2.imshow('Transformed Image', np.uint8(horizontally_stacked))
                cv2.waitKey()

            temp = transformed_data['bboxes']
            class_labels = transformed_data['class_labels']
            for i in range(len(temp)):
                annotation[i, 0:4] = temp[i]
                annotation[i, -1] = float(self.labels.index(class_labels[i]))

            return np.array(transformed_data['image']) / 255., annotation

        else:
            raise Exception('No valid type of network selected')

    def __augment_batch(self, img_batch, label_batch):

        if self.network_type == 'classification':
            for i in range(img_batch.shape[0]):
                img_batch[i, ...] = self.__random_transform(img_batch[i, ...], self.valid_augmentations)
            return img_batch

        elif self.network_type == 'regression':
            for i in range(img_batch.shape[0]):
                img_batch[i, ...] = self.__random_transform(img_batch[i, ...], self.valid_augmentations)
            return img_batch

        elif self.network_type == 'segmentation':
            for i in range(img_batch.shape[0]):
                img_batch[i, ...], label_batch[i, ...] = self.__random_transform(img_batch[i, ...], label_batch[i, ...])
            return img_batch, label_batch
        else:
            raise Exception('No valid type of network selected')

    def preprocess_true_boxes(self, true_boxes, input_shape, anchors, num_classes):
        # ToDo: Dry run this code at least once.
        '''Preprocess true boxes to training input format

        Parameters
        ----------
        true_boxes: array, shape=(batch_size, max boxes per img, 5)
            Absolute x_min, y_min, x_max, y_max, class_id relative to input_shape.
        input_shape: array-like, hw, multiples of 32
        anchors: array, shape=(N, 2), (9, wh)
        num_classes: int

        Returns
        -------
        y_true: list of array, shape like yolo_outputs, xywh are relative value

        '''

        num_stages = 3  # default setting for yolo, tiny yolo will be 2
        anchor_mask = [[0, 1, 2], [3, 4, 5], [6, 7, 8]]
        bbox_per_grid = 3
        true_boxes = np.array(true_boxes, dtype='float32')
        true_boxes_abs = np.array(true_boxes, dtype='float32')
        input_shape = np.array(input_shape, dtype='int32')
        true_boxes_xy = (true_boxes_abs[..., 0:2] + true_boxes_abs[..., 2:4]) // 2  # (100, 2)
        true_boxes_wh = true_boxes_abs[..., 2:4] - true_boxes_abs[..., 0:2]  # (100, 2)

        # Normalize x,y,w, h, relative to img size -> (0~1)
        true_boxes[..., 0:2] = true_boxes_xy / input_shape[::-1]  # xy
        true_boxes[..., 2:4] = true_boxes_wh / input_shape[::-1]  # wh

        bs = true_boxes.shape[0]
        grid_sizes = [input_shape // {0: 8, 1: 16, 2: 32}[stage] for stage in range(num_stages)]
        y_true = [np.zeros((bs,
                            grid_sizes[s][0],
                            grid_sizes[s][1],
                            bbox_per_grid,
                            5 + num_classes), dtype='float32')
                  for s in range(num_stages)]
        # [(?, 52, 52, 3, 5+num_classes) (?, 26, 26, 3, 5+num_classes)  (?, 13, 13, 3, 5+num_classes) ]
        y_true_boxes_xywh = np.concatenate((true_boxes_xy, true_boxes_wh), axis=-1)
        # Expand dim to apply broadcasting.
        anchors = np.expand_dims(anchors, 0)  # (1, 9 , 2)
        anchor_maxes = anchors / 2.  # (1, 9 , 2)
        anchor_mins = -anchor_maxes  # (1, 9 , 2)
        valid_mask = true_boxes_wh[..., 0] > 0  # (1, 100)

        for batch_idx in range(bs):
            # Discard zero rows.
            wh = true_boxes_wh[batch_idx, valid_mask[batch_idx]]  # (# of bbox, 2)
            num_boxes = len(wh)
            if num_boxes == 0: continue
            wh = np.expand_dims(wh, -2)  # (# of bbox, 1, 2)
            box_maxes = wh / 2.  # (# of bbox, 1, 2)
            box_mins = -box_maxes  # (# of bbox, 1, 2)

            # Compute IoU between each anchors and true boxes for responsibility assignment
            intersect_mins = np.maximum(box_mins, anchor_mins)  # (# of bbox, 9, 2)
            intersect_maxes = np.minimum(box_maxes, anchor_maxes)
            intersect_wh = np.maximum(intersect_maxes - intersect_mins, 0.)
            intersect_area = np.prod(intersect_wh, axis=-1)  # (9,)
            box_area = wh[..., 0] * wh[..., 1]  # (# of bbox, 1)
            anchor_area = anchors[..., 0] * anchors[..., 1]  # (1, 9)
            iou = intersect_area / (box_area + anchor_area - intersect_area)  # (# of bbox, 9)

            # Find best anchor for each true box
            best_anchors = np.argmax(iou, axis=-1)  # (# of bbox,)
            for box_idx in range(num_boxes):
                best_anchor = best_anchors[box_idx]
                for stage in range(num_stages):
                    if best_anchor in anchor_mask[stage]:
                        x_offset = true_boxes[batch_idx, box_idx, 0] * grid_sizes[stage][1]
                        y_offset = true_boxes[batch_idx, box_idx, 1] * grid_sizes[stage][0]
                        # Grid Index
                        grid_col = np.floor(x_offset).astype('int32')
                        grid_row = np.floor(y_offset).astype('int32')
                        anchor_idx = anchor_mask[stage].index(best_anchor)
                        class_idx = true_boxes[batch_idx, box_idx, 4].astype('int32')
                        y_true[stage][batch_idx, grid_row, grid_col, anchor_idx, :2] = true_boxes_xy[batch_idx, box_idx,
                                                                                       :]  # abs xy
                        y_true[stage][batch_idx, grid_row, grid_col, anchor_idx, 2:4] = true_boxes_wh[batch_idx,
                                                                                        box_idx,
                                                                                        :]  # abs wh
                        y_true[stage][batch_idx, grid_row, grid_col, anchor_idx, 4] = 1  # confidence

                        y_true[stage][batch_idx, grid_row, grid_col, anchor_idx, 5 + class_idx] = 1  # one-hot encoding

        return y_true, y_true_boxes_xywh


In [ ]:
class CustomCallback(tf.keras.callbacks.Callback):
    def __init__(self, model, network_type, validation_data, validation_labels, major_revision, minor_revision,
                 model_save_path,
                 model_log_path, model_name, use_validataion_accuracy, patience=10, map_iou_threshold = 0.5,
                 image_size = (608, 608), classes = []):
        self.model = model
        self.network_type = network_type
        self.validation_data = validation_data
        self.validation_labels = validation_labels
        self.patience = patience
        self.major_revision = major_revision
        self.minor_revision = minor_revision
        self.model_save_path = model_save_path
        self.model_log_path = model_log_path
        self.model_name = model_name
        self.use_validation_accuracy = use_validataion_accuracy
        self.accuracies = []
        self.map_iou_threshold = map_iou_threshold
        self.image_size = image_size
        self.my_classes = classes

    def on_train_begin(self, logs=None):
        file = open(os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                    'a+')
        file.write('#' * 25)
        file.write('\n')
        file.write('####MODEL NAME %s\n' % (self.model_name))

    def on_epoch_end(self, epoch, logs={}):
        """
        We will calculate the validation accuracy at the end of each epoch and then we will log that accuracy to a file.
        :param epoch: current epoch
        :param logs:
        :return: None
        """
        loss = logs.get('loss')
        if loss is not None:
            if np.isnan(loss) or np.isinf(loss):
                print('Epoch %d: Invalid loss, terminating training' % epoch)
                file = open(
                    os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                    'a+')
                file.write('Epoch %d: Invalid loss, terminating training' % epoch)
                self.model.stop_training = True

        if self.network_type == 'classification':
            if self.use_validation_accuracy:
                predictions = self.model.predict(self.validation_data)
                predictions = np.argmax(predictions, axis=1)

                validation_accuracy = accuracy_score(np.argmax(self.validation_labels[0:len(predictions), :], axis=1),
                                                     predictions, normalize=True)
                my_confusion_matrix = confusion_matrix(np.argmax(self.validation_labels[0:len(predictions), :], axis=1),
                                                       predictions)

                # Logging to file
                file = open(
                    os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                    'a+')
                file.write('#' * 25)
                file.write('\n')
                file.write('####EPOCH %i\n' % (epoch + 1))
                file.write('#### VALIDATION ACCURACY =%.5f \n' % validation_accuracy)
                file.write('####CONFUSION MATRIX####')
                file.write('\n')
                file.write(str(my_confusion_matrix))

                print('\n')
                print('#' * 25)
                print('#### EPOCH %i' % (epoch + 1))
                print('#### VALIDATION ACCURACY =%.5f' % validation_accuracy)
                print('####CONFUSION MATRIX####')
                print(str(my_confusion_matrix))
                print('#' * 25)

                self.accuracies.append(validation_accuracy)
                x = np.asarray(self.accuracies)
                if np.argsort(-x)[0] == (len(x) - self.patience - 1):
                    print('#### Validation accuracy no increase for %i epochs: EARLY STOPPING' % self.patience)
                    file.write('#### Validation accuracy no increase for %i epochs: EARLY STOPPING\n' % self.patience)
                    self.model.stop_training = True

                if (validation_accuracy > 0.000) & (validation_accuracy >= np.nanmax(self.accuracies)):
                    print('#### Saving new best...')
                    file.write('#### Saving new best...\n')
                    self.model.save_weights(
                        os.path.join(self.model_save_path, 'm%i-%i.h5' % (self.major_revision, self.minor_revision)))

                file.close()

            else:
                # Logging to file
                file = open(
                    os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                    'a+')
                file.write('#' * 25)
                file.write('\n')
                file.write('####EPOCH %i\n' % (epoch + 1))
                file.write('#### TRAINING ACCURACY =%.5f \n' % logs.get('categorical_accuracy'))

                self.accuracies.append(logs.get('categorical_accuracy'))
                x = np.asarray(self.accuracies)
                if np.argsort(-x)[0] == (len(x) - self.patience - 1):
                    print('#### Validation accuracy no increase for %i epochs: EARLY STOPPING' % self.patience)
                    file.write('#### Validation accuracy no increase for %i epochs: EARLY STOPPING\n' % self.patience)
                    self.model.stop_training = True

                if (logs.get('categorical_accuracy') > 0.000) & (logs.get('categorical_accuracy') >
                                                                 np.nanmax(self.accuracies)):
                    print('#### Saving new best...')
                    file.write('#### Saving new best...\n')
                    self.model.save_weights(
                        os.path.join(self.model_save_path, 'm%i-%i.h5' % (self.major_revision, self.minor_revision)))

                file.close()

        elif self.network_type == 'regression':
            if self.use_validation_accuracy:
                predictions = self.model.predict(self.validation_data)[:,0]

                mean_absolute_error = mae(self.validation_labels[0:len(predictions)], predictions)

                # Logging to file
                file = open(
                    os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                    'a+')
                file.write('#' * 25)
                file.write('\n')
                file.write('####EPOCH %i\n' % (epoch + 1))
                file.write('#### MEAN ABSOLUTE ERROR =%.5f \n' % mean_absolute_error)

                print('\n')
                print('#' * 25)
                print('#### EPOCH %i' % (epoch + 1))
                print('#### MEAN ABSOLUTE ERROR =%.5f' % mean_absolute_error)
                print('#' * 25)

                self.accuracies.append(mean_absolute_error)
                x = np.asarray(self.accuracies)
                if np.argsort(-x)[0] == (len(x) - self.patience - 1):
                    print('#### Validation accuracy no increase for %i epochs: EARLY STOPPING' % self.patience)
                    file.write('#### Validation accuracy no increase for %i epochs: EARLY STOPPING\n' % self.patience)
                    self.model.stop_training = True

                if (mean_absolute_error > 0.000) & (mean_absolute_error <= np.nanmin(self.accuracies)):
                    print('#### Saving new best...')
                    file.write('#### Saving new best...\n')
                    self.model.save_weights(
                        os.path.join(self.model_save_path, 'm%i-%i.h5' % (self.major_revision, self.minor_revision)))

                file.close()

            else:
                # Logging to file
                file = open(
                    os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                    'a+')
                file.write('#' * 25)
                file.write('\n')
                file.write('####EPOCH %i\n' % (epoch + 1))
                file.write('#### TRAINING ACCURACY =%.5f \n' % logs.get('categorical_accuracy'))

                self.accuracies.append(logs.get('mae'))
                x = np.asarray(self.accuracies)
                if np.argsort(-x)[0] == (len(x) - self.patience - 1):
                    print('#### Validation accuracy no increase for %i epochs: EARLY STOPPING' % self.patience)
                    file.write('#### Validation accuracy no increase for %i epochs: EARLY STOPPING\n' % self.patience)
                    self.model.stop_training = True

                if (logs.get('mae') > 0.000) & (logs.get('mae') <=
                                                                 np.nanmin(self.accuracies)):
                    print('#### Saving new best...')
                    file.write('#### Saving new best...\n')
                    self.model.save_weights(
                        os.path.join(self.model_save_path, 'm%i-%i.h5' % (self.major_revision, self.minor_revision)))

                file.close()



        elif self.network_type == 'segmentation':
            my_mean_average_precision = calculate_map(validation_samples_list = self.validation_data,
                                                      yolo_model =WorkAround.current_model , iou_threshold=self.map_iou_threshold,
                                                      model_image_size = self.image_size,
                                                      model_classes = self.my_classes)
            file = open(os.path.join(self.model_log_path, 'log-%i-%i.txt' % (self.major_revision, self.minor_revision)),
                        'a+')
            file.write('#' * 25)
            file.write('\n')
            file.write('####EPOCH %i\n' % (epoch + 1))
            file.write('#### TRAINING LOSS =%.5f \n' % logs.get('loss'))
            file.write('#### VALIDATION LOSS =%.5f \n' % logs.get('val_loss'))
            file.write('#### mAP =%.5f \n' % my_mean_average_precision)

            print()
            print('mAP at ', self.map_iou_threshold, ' is: ', my_mean_average_precision)
            temp_monitored_value = logs.get('val_loss')
            if not math.isnan(temp_monitored_value):
                self.accuracies.append(temp_monitored_value)
            else:
                self.accuracies.append(1000000)

            x = np.asarray(self.accuracies)
            if len(x) > 1:
                if np.argsort(x)[0] == (len(x) - self.patience - 1):
                    print('#### Validation accuracy no increase for %i epochs: EARLY STOPPING' % self.patience)
                    file.write('#### Validation accuracy no increase for %i epochs: EARLY STOPPING\n' % self.patience)
                    self.model.stop_training = True

                if (temp_monitored_value > 0.000) & (temp_monitored_value < np.min(self.accuracies[:-1])):
                    print('#### Saving new best...')
                    file.write('#### Saving new best...\n')
                    WorkAround.save_inference_model(path=self.model_save_path, major_revision=self.major_revision,
                                                    minor_revision=self.minor_revision)

            file.close()

        else:
            raise Exception('No valid network type selected')


In [ ]:
def alternate_learning_rate(epoch):
    """
        An alternate scheduler for each epoch.
        :param epoch: current epoch
        :return: learning rate for current epoch
        """
    if epoch % 2 == 0:
        return 1e-5
    else:
        return 1e-4

In [ ]:
def cosine_annealing_scheduler(epoch):
    """
        A step rate scheduler for setting the learning rate for each epoch.
        :param epoch: current epoch
        :return: learning rate for current epoch
        """
    learning_rate_min = 1e-6
    learning_rate_max = 1e-3
    epochs_per_cycle = 20

    return learning_rate_min + (learning_rate_max - learning_rate_min) * \
           (1 + math.cos(math.pi * (epoch % epochs_per_cycle) / epochs_per_cycle)) / 2




In [ ]:
def constant_learning_scheduler(epoch):
    """
    Returns a constant learning rate for every epoch
    :param epoch: current epoch
    :return: learning rate
    """
    return 1e-5

In [ ]:
def learning_rate_scheduler(epoch):
    """
    For training classification models we first use a step rate scheduler for initial 50 epochs and then switch
    to cosine annealing scheduler for the remaining epochs.
    This has the benefit of starting the learning rate slowly and then having the benefits of a cylic scheduler.
    :param epoch: current epoch
    :return: learning rate
    """
    if epoch < 10:
        return 0.000001
    elif epoch < 50:
        return step_rate_scheduler(epoch)
    else:
        return cosine_annealing_scheduler(epoch)

In [ ]:
def step_rate_scheduler(epoch):
    """
    A step rate scheduler for setting the learning rate for each epoch.
    :param epoch: current epoch
    :return: learning rate for current epoch
    """
    learning_rate_start = 1e-5
    learning_rate_max = 1e-3
    learning_rate_rampup_epochs = 5
    Learning_rate_sustain_epoch = 0
    learning_rate_step_decay = 0.75

    if epoch < learning_rate_rampup_epochs:
        lr = (learning_rate_max - learning_rate_start) / learning_rate_rampup_epochs * epoch + learning_rate_start
    elif epoch < learning_rate_rampup_epochs + Learning_rate_sustain_epoch:
        lr = learning_rate_max
    else:
        lr = learning_rate_max * \
             learning_rate_step_decay ** ((epoch - learning_rate_rampup_epochs - Learning_rate_sustain_epoch) // 10)

    return lr

In [ ]:
def generate_image_paths_and_labels(network_type, input_dir, split, read_from_folders, file_name, classes):
    """
    For Classification:
        Function to read images names with paths from multiple folders and create their labels according to
        folder name and then store them in a readable file that is used the next time around.
        The function first checks for a single .csv file with the name file_name that is created on the first pass.
        If that file exists, then that file is directly loaded. If the file doesn't exist, then all the images are read.

    For Segmentation:
        Function looks for a single .txt file with the name file_name containing the paths to all the iamges and their
        corresponding bounding boxes and labels. If no such file is found, then an exception is raised.
        If .txt file exists, then it is read and total samples are split into training and testiong samples according
        to the given split.
    return: Training and testing samples names with paths and labels.
    """
    my_dataframe = None
    if network_type == 'classification':
        image_extensions = ['.jpg', '.jpeg', '.png', '.JPG']
        images = []
        labels = []

        if read_from_folders:
            print('Reading image names from folders...')
            directories = [x[0] for x in os.walk(input_dir)]
            for directory in tqdm(directories, desc='Directories Done'):
                for each_class in classes:
                    if each_class in directory:
                        files = os.listdir(directory)
                        files = [file for file in files if
                                 any(image_extension in file for image_extension in image_extensions)]
                        images.extend([os.path.join(directory, file) for file in files])
                        labels.extend([classes.index(each_class)] * len(files))

            # Saving files as npz array for future reads
            my_dict = {}
            my_dict['id'], my_dict['label'] = images, labels
            my_dataframe = pd.DataFrame(data=my_dict, index=None)
            my_dataframe.to_csv(os.path.join(input_dir, file_name), sep=',', index=False)
            unique_labels = np.unique(labels)
            total_images = 0
            for unique_label in unique_labels:
                total_images += np.count_nonzero(np.where(np.array(labels) == unique_label))
                print('Total Images of Class ' + str(unique_label + 1) + ': ',
                      np.count_nonzero(np.where(np.array(labels) == unique_label)))
            print('Total Images Read: ', total_images)

        else:
            path_and_label_file = os.path.exists(os.path.join(input_dir, file_name))

            if path_and_label_file:
                print('Reading the .csv file')

                my_dataframe = pd.read_csv(os.path.join(input_dir, file_name), sep=',')
                images, labels = my_dataframe['id'].to_list(), my_dataframe['label'].to_list()
                unique_labels = np.unique(labels)
                total_images = 0
                for unique_label in unique_labels:
                    total_images += np.count_nonzero(np.where(np.array(labels) == unique_label))
                    print('Total Images of Class ' + str(unique_label + 1) + ': ',
                          np.count_nonzero(np.where(np.array(labels) == unique_label)))
                print('Total Images Read: ', total_images)

            else:
                print('Relevant .csv files do not exist. Data set not loaded.')


    elif network_type == 'segmentation':
        text_file = os.path.exists(os.path.join(input_dir, file_name))

        if text_file:
            print('Reading the .txt file')
            my_dataframe = pd.read_csv(os.path.join(input_dir, file_name), sep='\n', header=None)
            print('Total Examples: ', len(my_dataframe))
            examples = my_dataframe[0].to_list()

            examples = shuffle(examples)

            training, testing, training_labels, testing_labels = train_test_split(examples, [0] * len(examples),
                                                                                  train_size=split)
            return training, training_labels, testing, testing_labels

        else:
            raise Exception('Text file with segmentation examples does not exist')

    elif network_type == 'regression':
        path_and_label_file = os.path.exists(os.path.join(input_dir, file_name))

        if path_and_label_file:
            print('Reading the .csv file')

            my_dataframe = pd.read_csv(os.path.join(input_dir, file_name), sep=',')
            images, labels = my_dataframe['id'].to_list(), my_dataframe['label'].to_list()
            total_images = len(images)
            print('Total Images Read: ', total_images)

        else:
            print('Relevant .csv files do not exist. Data set not loaded.')

    else:
        raise Exception('No valid network type selected')

    images, labels = shuffle(images, labels)

    training, testing, training_labels, testing_labels = train_test_split(images, labels, train_size=split)

    return training, training_labels, testing, testing_labels

In [ ]:
print(len(os.listdir('../input/dicom-image-extractor/')))


In [ ]:
!mkdir  './1/'
!mkdir  './2/'
#!mkdir  './3/'
#!mkdir  './4/'
!mkdir  './models/'
!mkdir  './logs/'

In [ ]:
#Separating SIIM train images for classification
my_csv = pd.read_csv('../input/siim-covid19-detection/train_study_level.csv', sep=',')
input_dir = '../input/dicom-image-extractor/'
output_dir = './'

all_images = os.listdir(input_dir)

possible_classes = ['1', '2', '3', '4']

for row in tqdm(range(len(my_csv))):
    my_row = my_csv.loc[row].values.tolist()
    image_name = my_row[0].split('_')

    complete_name = [x for x in all_images if image_name[0] in x]
    #print(complete_name)
    #if len(complete_name) > 1:
        #print(complete_name)

        
    for value in complete_name:

        my_image = cv2.imread(os.path.join(input_dir, value), -1)
        #print(np.shape(my_image))

        #my_image = np.stack((my_image,)*3, axis=-1)

        if my_row[1] == 1:
            #print(os.path.join(output_dir, possible_classes[0], value))
            cv2.imwrite(os.path.join(output_dir, possible_classes[0], value), my_image)
            #print(len(os.path.join(output_dir, possible_classes[0])))

        elif my_row[2] == 1:
            cv2.imwrite(os.path.join(output_dir, possible_classes[1], value), my_image)
            #print(len(os.path.join(output_dir, possible_classes[1])))

        elif my_row[3] == 1:
            cv2.imwrite(os.path.join(output_dir, possible_classes[1], value), my_image)
            #print(len(os.path.join(output_dir, possible_classes[2])))

        elif my_row[4] == 1:
            cv2.imwrite(os.path.join(output_dir, possible_classes[1], value), my_image)
            #print(len(os.path.join(output_dir, possible_classes[3])))

        else:
            pass

print('Done')

In [ ]:
print(len(os.listdir('./1')))
print(len(os.listdir('./2')))
#print(len(os.listdir('./3')))
#print(len(os.listdir('./4')))

In [ ]:
input_dim = (600, 600, 3)
# Training Parameters:
epochs, initial_epoch = 14, 0
network_type, batch_size, n_classes, model_name, build_new, patience = "classification", 2, 4, "EfficientNetB7", True, 15
major_revision, minor_revision = 2, 41
no_improvement = False

In [ ]:
# Data Generator Parameters
train_using_generator, shuffle_data, augment_data, return_labels, visualise = True, True, True, True, False
training_testing_split = 0.9
read_from_folder = True
#classes = ['aortic_enlargement', 'atelectasis', 'calcification', 'cardiomegaly',
#           'consolidation', 'ild', 'infiltration', 'lung_opacity', 'no_finding', 'nodule', 'other_lesion',
#           'pleural_effusion', 'pleural_thickening', 'pneumothorax', 'pulmonary_fibrosis']
classes = ['1', '2']
file_name = 'image_paths_and_labels_updated.csv'

In [ ]:
p = 0.25
augmentations = [albu.Flip(p=p),albu.HorizontalFlip(p=p),albu.ShiftScaleRotate(shift_limit=(-0.1599999964237213, 0.05999999865889549),scale_limit=(-0.10000000149011612, 0.05000000074505806),rotate_limit=(-39, -7),interpolation=0, border_mode=0, value=(0, 0, 0), mask_value=None, p=p),
                     albu.RandomBrightness(p=p), albu.Rotate(p=p, limit=90),
                     albu.RandomContrast(limit=0.5, p=p),
                     albu.NoOp(p=p), 0.65]
use_validation_data = True

In [ ]:
def focal_loss(gamma=2., alpha=1.):
    """Implements focal loss for the deep learning model"""

    gamma = float(gamma)
    alpha = float(alpha)

    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, - tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [ ]:
input_dir = "./"
model_save_path = "./models"
log_save_path = "./logs"

In [ ]:
# Data Generator Parameters
train_using_generator, shuffle_data, augment_data, return_labels, visualise = True, True, True, True, False
training_testing_split = 0.9
read_from_folder = True

In [ ]:
training_data, training_labels, testing_data, testing_labels = generate_image_paths_and_labels(network_type=network_type, input_dir=input_dir, split=training_testing_split, read_from_folders=read_from_folder, file_name=file_name, classes=classes)

In [ ]:
my_input_layer = tf.keras.Input(shape=input_dim)
my_base_model = efn.EfficientNetB7(weights='imagenet', include_top=False, input_shape=input_dim)
x = my_base_model(my_input_layer)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(8, activation='relu', name='Dense_1', dtype='float32')(x)
x = tf.keras.layers.Dense(n_classes, activation='softmax', name='Output', dtype='float32')(x)
my_model = tf.keras.Model(inputs=my_input_layer, outputs=x, name=model_name)
my_optimiser = tf.keras.optimizers.Adam(lr=0.00001)
#my_model.compile(loss='categorical_crossentropy', optimizer=my_optimiser, metrics=['categorical_accuracy'])
my_model.compile(loss=focal_loss(), optimizer=my_optimiser,metrics=['accuracy'])
my_model.summary()
my_model.save(os.path.join('./models', 'm%i-%i_%s.h5' % (major_revision, minor_revision, 'model')))

In [ ]:
# my_input_layer = tf.keras.Input(shape=input_dim)
# my_base_model = efn.EfficientNetB7(weights='imagenet', include_top=False, input_shape=input_dim)
# x = my_base_model(my_input_layer)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# x = tf.keras.layers.Dense(8, activation='relu', name='Dense_1', dtype='float32')(x)
# x = tf.keras.layers.Dense(n_classes, activation='softmax', name='Output', dtype='float32')(x)
# my_model = tf.keras.Model(inputs=my_input_layer, outputs=x, name=model_name)
# my_optimiser = tf.keras.optimizers.Adam(lr=0.00001)
# my_model.compile(loss='categorical_crossentropy', optimizer=my_optimiser, metrics=['categorical_accuracy'])
# #my_model.compile(loss=focal_loss(), optimizer=my_optimiser,metrics=['accuracy'])
# my_model.summary()
# my_model.save(os.path.join('./models', 'm%i-%i_%s.h5' % (major_revision, minor_revision, 'model')))

In [ ]:
generated_train_data = DataGenerator(list_ids=training_data, labels=training_labels, return_labels=return_labels, batch_size=batch_size, dim=input_dim,n_classes=n_classes, shuffle_data=shuffle_data, augment_data=augment_data,network_type=network_type, visualise=visualise,major_revision=major_revision, minor_revision=minor_revision,train_on_cloud=False,model_log_path=log_save_path,augmentations=augmentations)

generated_validation_data = DataGenerator(list_ids=testing_data, labels=testing_labels, return_labels=False, batch_size=batch_size, dim=input_dim,n_classes=n_classes, shuffle_data=False,augment_data=False,network_type=network_type, visualise=visualise,major_revision=major_revision,minor_revision=minor_revision, model_log_path=log_save_path,augmentations=augmentations)

testing_labels = to_categorical(testing_labels, num_classes=n_classes)

In [ ]:
 # Creating Callbacks
my_custom_callback = CustomCallback(model=my_model, network_type=network_type, validation_data=generated_validation_data, validation_labels=testing_labels, major_revision=major_revision, minor_revision=minor_revision,model_save_path=model_save_path, model_log_path=log_save_path,model_name=model_name, use_validataion_accuracy=use_validation_data,patience=patience)

my_callbacks = [my_custom_callback,tf.keras.callbacks.LearningRateScheduler(alternate_learning_rate, verbose=True)]

In [ ]:
print('Scheduler selected: ', alternate_learning_rate)

print('Training Testing Split is: ', training_testing_split)

In [ ]:
my_model_history = my_model.fit(generated_train_data, epochs=epochs,initial_epoch=initial_epoch, callbacks=my_callbacks, verbose=1, max_queue_size=(1 * 1) * 64, workers=128)

In [ ]:
!rm -r './1/'
!rm -r './2/'
!rm -r './3/'
!rm -r './4/'